In [1]:
# Largely based off this example
# http://nbviewer.jupyter.org/github/kyper-data/python-highcharts/blob/master/examples/ipynb/highstock/flags-general-2.ipynb

In [2]:
%reload_ext autoreload
%autoreload 2

import os
from datetime import datetime
import numpy as np
from IPython.display import display, display_pretty, Javascript, HTML
from highcharts import Highstock
import matplotlib.pyplot as plt
from lxml import etree
import urllib2

In [16]:
x_range = '20170307&end=' + datetime.now().strftime("%Y%m%d")

In [17]:
# Extract historical www.coinmarketcap.com data for XRP
ripple_url = r'https://coinmarketcap.com/currencies/ripple/historical-data/?start=' + x_range
response = urllib2.urlopen(ripple_url)
html_parser = etree.HTMLParser()
tree = etree.parse(response, html_parser)

# Select the data using xpath from the html
header = tree.xpath(
    '//div[@id="historical-data"]/div[@class="tab-header"]/div[@class="table-responsive"]/table/thead/tr/th/text()')
rows = tree.xpath(
        '//div[@id="historical-data"]/div[@class="tab-header"]/div[@class="table-responsive"]/table/tbody/tr')
# Create table from list of row selector objects
table = list(reversed([row.xpath('./td/text()') for row in rows])) # table is a list of lists
date = [datetime.strptime(row[0], '%b %d, %Y') for row in table]
xrp_open = [float(row[1]) for row in table]
xrp_high = [float(row[2]) for row in table]
xrp_low = [float(row[3]) for row in table]
xrp_close = [float(row[4]) for row in table]
xrp_volume = [int(row[5].replace('-','0').replace(',','')) for row in table]
xrp_marketcap = [int(row[6].replace(',','')) for row in table]

In [18]:
print xrp_close[:10]

[0.006534, 0.006509, 0.006498, 0.006227, 0.006238, 0.006255, 0.006385, 0.006381, 0.006314, 0.006394]


In [19]:
bitcoin_url = r'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=' + x_range
response = urllib2.urlopen(bitcoin_url)
html_parser = etree.HTMLParser()
tree = etree.parse(response, html_parser)
rows = tree.xpath(
        '//div[@id="historical-data"]/div[@class="tab-header"]/div[@class="table-responsive"]/table/tbody/tr')

# Create table from list of row selector objects
table = list(reversed([row.xpath('./td/text()') for row in rows])) # table is a list of lists
btc_close = [float(row[4]) for row in table]

In [20]:
print btc_close[:10]

[1223.54, 1150.0, 1188.49, 1116.72, 1175.83, 1221.38, 1231.92, 1240.0, 1249.61, 1187.81]


In [21]:
xrp_btc_price = [round(xrp_close[i] * 100000000 / btc_close[i],0) for i in range(len(btc_close))]

In [22]:
price_data = [list(tuble) for tuble in zip(date, xrp_btc_price)]

In [54]:
H = Highstock()
H.add_data_set(price_data, 'line', 'Price (Sats)', id = 'dataseries')

options = {
    'chart' : {
        'height' : 650
    },

    'title' : {
        'text' : 'Psychology of Ripple (XRP) market cycle... How to get rekt'
    },
    'tooltip': {
                'style': {
                    'width': '200px'
                },
                'valueDecimals': 0,
                'shared' : True
            },

    'yAxis' : {
        'title' : {
            'text' : "Price  (BTC - Satoshi's)"
        }
    },

}

H.set_dict_options(options)

data2 = [{
        'x' : datetime(2017, 3, 31),
        'title' : 'Disbelief',
        'text' : 'This rally will fail like the others'
    }, {
        'x' : datetime(2017, 4, 28),
        'title' : 'Hope',
        'text' : 'A recovery is possible'
    }, {
        'x' : datetime(2017, 5, 6),
        'title' : 'Optimism',
        'text' : 'This rally is real'
    }, {
        'x' : datetime(2017, 5, 8),
        'title' : 'Belief',
        'text' : 'Time to get fully invested'
    }, {
        'x' : datetime(2017, 5, 15),
        'title' : 'Thrill',
        'text' : 'I will buy more on margin, Gotta tell everyone to buy'
    }, {
        'x' : datetime(2017, 5, 17),
        'title' : 'Euphoria',
        'text' : 'I am a genious! We\'re all going to be rich'
    },{
        'x' : datetime(2017, 6, 3),
        'title' : 'Complacency',
        'text' : 'We just need to cool off for the next rally'
    }, {
        'x' : datetime(2017, 7, 3),
        'title' : 'Anxiety',
        'text' : 'Why am I getting margin calls? This dip is taing longer than expected'
    }, {
        'x' : datetime(2017, 7, 14),
        'title' : 'Denial',
        'text' : 'My investments are will great companies.They will come back'
    }, {
        'x' : datetime(2017, 7, 21),
        'title' : 'Panic',
        'text' : 'Shit! Everyone is selling. I need to get out!'
    }, {
        'x' : datetime(2017, 8, 5),
        'title' : 'Capitulation',
        'text' : 'I\'m getting 100% out of the margets. I can\'t afford to lose more'
    }, {
        'x' : datetime(2017, 8, 17),
        'title' : 'Anger',
        'text' : 'Who shorted the market?? Why did the government all this to happen??'
    },{
        'x' : datetime(2017, 11, 1),
        'title' : 'Depression',
        'text' : 'My retirement is lost. How can we pay for all this new stuff? I am an idiot.'
    }]


H.add_data_set(data2, 'flags', onSeries = 'dataseries',
                
                width = 12)

H